In [1]:
# Running %env without any arguments
# lists all environment variables

# The line below sets the environment
# variable CUDA_VISIBLE_DEVICES
%env CUDA_VISIBLE_DEVICES = 1

import numpy as np
import pandas as pd
import io
import time
import bson                       # this is installed with the pymongo package
import matplotlib.pyplot as plt
from scipy.misc import imread, imsave, imshow
import tensorflow as tf
from tensorflow.python.platform import tf_logging
from tensorflow.contrib import layers
from tensorflow.contrib.training import add_gradients_summaries
from tensorflow.python.ops import math_ops
from tensorflow.python.framework import ops
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import control_flow_ops
from tensorflow.python.training import optimizer as tf_optimizer
from tensorflow.python.ops import variables as tf_variables
import os.path
import tensorflow.contrib.slim as slim
import inception_preprocessing
import logging
import resnet1

# This is a bit of magic to make matplotlib figures appear inline in the notebook
# rather than in a new window.
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

env: CUDA_VISIBLE_DEVICES=1


In [2]:
DATASET_PATH = '/media/rs/0E06CD1706CD0127/Kapok/kaggle/'
PRETRAINED_MODEL_PATH = DATASET_PATH + 'Resnet/resnet_v1_50.ckpt'
LOG_PATH = DATASET_PATH + 'Resnet/logs/'
TRAIN_PATH = DATASET_PATH + 'Split1/Train/'
VAL_PATH = DATASET_PATH + 'Split1/Validation/'
TEST_PATH = DATASET_PATH + 'Test/'
CATEGORY_NAME_PATH = DATASET_PATH + 'category_names.csv'
BATCH_SIZE = 128#256

# total_batch_size is BATCH_SIZE * ACCUMULATE_STEP
ACCUMULATE_STEP = 2

IMAGE_WIDTH = 180
IMAGE_HEIGHT = 180
NUM_CLASS = 5270
LEVEL0_CLASS = 49
LEVEL1_CLASS = 483
# validation examples num: 2319624
# train examples num: 10051704
# total step: 157057
TOTAL_EXAMPLES = 10051704
VAL_EXAMPLES = 2319624
VAL_CHECK_FREQ = 50
NUM_EPOCHES = 8
VAL_NUM_EPOCHES = int(NUM_EPOCHES/(VAL_CHECK_FREQ*VAL_EXAMPLES/TOTAL_EXAMPLES)) + 1
EPOCHES_OVER = 4

INPUT_THREADS = 12

#Learning rate information and configuration (Up to you to experiment)
# initial_learning_rate = 0.000003#0.00001
# learning_rate_decay_factor = 0.94
initial_learning_rate = 0.0001#0.0004
learning_rate_decay_factor = 0.8
num_epochs_before_decay = 1
moving_average_decay = 0.9
momentum = 0.8
#Know the number steps to take before decaying the learning rate and batches per epoch
num_steps_per_epoch = TOTAL_EXAMPLES / (BATCH_SIZE * ACCUMULATE_STEP) + 1
decay_steps = int(num_epochs_before_decay * num_steps_per_epoch / 6)

In [3]:
# get TF logger
log = logging.getLogger('tensorflow')
log.setLevel(logging.DEBUG)

# create formatter and add it to the handlers
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# create file handler which logs even debug messages
fh = logging.FileHandler(DATASET_PATH + 'tensorflow_resnet_train.log')
fh.setLevel(logging.DEBUG)
fh.setFormatter(formatter)
log.addHandler(fh)

In [4]:
def preprocess_for_inception(input_image, is_training = True):
    return inception_preprocessing.preprocess_image(input_image, 160, 160, is_training)

In [5]:
class LabelMapping(object):
    def __init__(self, catogory_file_path):
        super(LabelMapping, self).__init__()
        self._category_level_csv = catogory_file_path
        self._category_map, self._category_level0_map, self._category_level1_map, self._len_level0, self._len_level1 = self.cvt_csv2tfrecord()
        self._mapping_strings = tf.constant( [ str(key) for key in self._category_map.keys() ] )
        #print(list(self._category_map.keys())[0])
        self._mapping_table = tf.contrib.lookup.index_table_from_tensor(mapping=self._mapping_strings, default_value=0) 
        
        self._level0_table = tf.contrib.lookup.HashTable(tf.contrib.lookup.KeyValueTensorInitializer(list(self._category_level0_map.keys()), list(self._category_level0_map.values()), tf.int64, tf.int64), 0)
        self._level1_table = tf.contrib.lookup.HashTable(tf.contrib.lookup.KeyValueTensorInitializer(list(self._category_level1_map.keys()), list(self._category_level1_map.values()), tf.int64, tf.int64), 0)

    @property
    def category_map(self):
        return self._category_map
    @property
    def level0_table(self):
        return self._level0_table
    @property
    def level1_table(self):
        return self._level1_table
    @property
    def len_level0(self):
        return self._len_level0
    @property
    def len_level1(self):
        return self._len_level1
    @property
    def mapping_table(self):
        return self._mapping_table
    
    def cvt_csv2tfrecord(self):
        level0_map, level1_map = self.create_level_map()
        count = 0
        category_map = dict()
        category_level0_map = dict()
        category_level1_map = dict()
        csv = pd.read_csv(self._category_level_csv).values
        for row in csv:  
            category_id, level0, level1 = row[0], row[1], row[2]
            category_map[category_id] = count
            category_level0_map[int(category_id)] = level0_map[level0]
            category_level1_map[int(category_id)] = level1_map[level1]
            count += 1

        return category_map, category_level0_map, category_level1_map, len(level0_map), len(level1_map)

    def create_level_map(self):
        csv = pd.read_csv(self._category_level_csv).values
        level_list = [list(), list()]
        for row in csv: 
            for level in range(1,3):
                if row[level] not in level_list[level-1]:
                    level_list[level-1].append(row[level])
        return dict(zip(level_list[0], range(len(level_list[0])))), dict(zip(level_list[1], range(len(level_list[1]))))

In [6]:
# class CdiscountDataset(object):
#     def __init__(self, data_path, file_begin_match, label_mapping, num_examples, num_classes, buffer_size, batch_size, num_epochs, is_training):
#         super(CdiscountDataset, self).__init__()
#         self._data_file_list = [ os.path.join(data_path, x) for x in os.listdir(data_path) if lambda x: os.path.isfile(x) and x.startswith(file_begin_match) ]
#         self._num_examples = num_examples
#         self._num_classes = num_classes
#         self._batch_size = batch_size
#         self._buffer_size = buffer_size
#         self._num_epochs = num_epochs
#         self._is_training = is_training
#         self._category_map = label_mapping.category_map
#         self._level0_table = label_mapping.level0_table
#         self._level1_table = label_mapping.level1_table
#         self._len_level0 = label_mapping.len_level0
#         self._len_level1 = label_mapping.len_level1
# #         print(self._len_level0)
# #         print(self._len_level1)
#         self._mapping_table = label_mapping.mapping_table
    
#     def _parse_function(self, example_proto):
#         features = {'img_raw': tf.FixedLenFeature([], tf.string, default_value=''),
#             'product_id': tf.FixedLenFeature([], tf.int64, default_value=tf.zeros([], dtype=tf.int64)),
#             'category_id': tf.FixedLenFeature([], tf.int64, default_value=tf.zeros([], dtype=tf.int64))}
                
#         parsed_features = tf.parse_single_example(example_proto, features)
#         image = preprocess_for_inception(tf.image.decode_image(parsed_features["img_raw"]), self._is_training)
#         raw_label = parsed_features["category_id"]
#         #raw_label = tf.constant(1000018736, dtype=tf.int64)
#         #image = tf.image.decode_image(parsed_features["img_raw"])
        
#         return image, tf.one_hot(self._mapping_table.lookup(tf.as_string(raw_label)), self._num_classes, axis=-1),\
#                 tf.one_hot(self._level0_table.lookup(raw_label), self._len_level0, axis=-1),\
#                 tf.one_hot(self._level1_table.lookup(raw_label), self._len_level1, axis=-1)
    
#     def get_next(self):
#         #next_example, next_label, next_level0_label, next_level1_label 
#         return self._iterator.get_next()
#     def create_dataset(self):
#         self._dataset = tf.data.TFRecordDataset(self._data_file_list, compression_type='ZLIB', buffer_size = 409600)
#         parse_func = lambda example : self._parse_function(example)
#         self._dataset = self._dataset.map(parse_func)
#         self._dataset = self._dataset.shuffle(buffer_size=self._buffer_size)
#         self._dataset = self._dataset.batch(self._batch_size)
#         self._dataset = self._dataset.repeat(self._num_epochs)
#         self._iterator = self._dataset.make_initializable_iterator()
        
#         return self._iterator.initializer
    
    
class CdiscountDataset(object):
    def __init__(self, data_path, file_begin_match, label_mapping, num_examples, num_classes, buffer_size, batch_size, num_epochs, is_training):
        super(CdiscountDataset, self).__init__()
        #self._data_file_list = [ os.path.join(data_path, x) for x in os.listdir(data_path) if lambda x: os.path.isfile(x) and x.startswith(file_begin_match) ]
        self._data_file_list = data_path + file_begin_match + '*'
        self._num_examples = num_examples
        self._num_classes = num_classes
        self._batch_size = batch_size
        self._buffer_size = buffer_size
        self._num_epochs = num_epochs
        self._is_training = is_training
        self._category_map = label_mapping.category_map
        self._level0_table = label_mapping.level0_table
        self._level1_table = label_mapping.level1_table
        self._len_level0 = label_mapping.len_level0
        self._len_level1 = label_mapping.len_level1
        self._mapping_table = label_mapping.mapping_table
    
    
    def create_dataset(self):
        opts = tf.python_io.TFRecordOptions(tf.python_io.TFRecordCompressionType.ZLIB)
        reader = lambda : tf.TFRecordReader(options=opts)
        keys_to_features = {
            'img_raw': tf.FixedLenFeature([], tf.string, default_value=''),
            'product_id': tf.FixedLenFeature([], tf.int64, default_value=tf.zeros([], dtype=tf.int64)),
            # notice that we don't have this feature in our TFRecord, so always default provided
            'format': tf.FixedLenFeature([], tf.string, default_value='jpg'),
            'category_id': tf.FixedLenFeature([], tf.int64, default_value=tf.zeros([], dtype=tf.int64))
        }

        items_to_handlers = {
            # automated decode image from features in FixedLenFeature
            'image': slim.tfexample_decoder.Image(image_key='img_raw', format_key='format'),
            'label': slim.tfexample_decoder.Tensor('category_id'),
        }

        decoder = slim.tfexample_decoder.TFExampleDecoder(keys_to_features, items_to_handlers)
        
        self._dataset = slim.dataset.Dataset(
            data_sources = self._data_file_list,
            decoder = decoder,
            reader = reader,
            # num_readers = 8,
            num_samples = self._num_examples,
            #num_classes = self._num_classes,
            items_to_descriptions = None)
        
        # notice that DatasetDataProvider can automate shuffle the examples by ParallelReader using its RandomShuffleQueue
        self._data_provider = slim.dataset_data_provider.DatasetDataProvider(
            self._dataset,
            num_readers = INPUT_THREADS,
            shuffle = True, # default is True
            num_epochs = self._num_epochs,
            common_queue_capacity = self._buffer_size + 4 * self._batch_size,
            common_queue_min = self._buffer_size,
            scope = self._is_training and 'train_files' or 'validation_files')
        
        org_image, org_label = self._data_provider.get(['image', 'label'])

        image = preprocess_for_inception(org_image, self._is_training) # final image to train
 
        # no need for shuffle, DatasetDataProvider do this for us
        batch_images, batch_labels, batch_labels_level0, batch_labels_level1 = \
                tf.train.batch([image, tf.one_hot(self._mapping_table.lookup(tf.as_string(org_label)), self._num_classes, axis=-1),\
                tf.one_hot(self._level0_table.lookup(org_label), self._len_level0, axis=-1),\
                tf.one_hot(self._level1_table.lookup(org_label), self._len_level1, axis=-1)],\
                self._batch_size,\
                num_threads = INPUT_THREADS,\
                capacity = self._buffer_size + 4 * self._batch_size,\
                allow_smaller_final_batch = self._is_training, name = self._is_training and 'train_batch' or 'validation_batch')
        
        return batch_images, batch_labels, batch_labels_level0, batch_labels_level1

In [7]:
# # Parameters
# learning_rate = 0.01
# total_batch = 25
# batch_size = 4
# display_step = 1
# BATCH_SIZE = 2
# # total_batch_size is BATCH_SIZE * ACCUMULATE_STEP
# ACCUMULATE_STEP = 2
# accumulate_factor = tf.constant([1./ACCUMULATE_STEP])

# # tf Graph Input
# x = tf.placeholder(tf.float32, [None, 3]) # mnist data image of shape 28*28=784
# y = tf.placeholder(tf.float32, [None, 4]) # 0-9 digits recognition => 10 classes

# # Set model weights
# W = tf.Variable(tf.zeros([3, 4]))
# b = tf.Variable(tf.zeros([4]))

# # Construct model
# pred = tf.nn.softmax(tf.matmul(x, W) + b) # Softmax

# global_step = tf.train.get_or_create_global_step()

# # Minimize error using cross entropy
# total_loss = tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred), reduction_indices=1))
# # Gradient Descent
# optimizer = tf.train.GradientDescentOptimizer(learning_rate)#.minimize(cost, global_step=global_step)

# variables_to_train = tf_variables.trainable_variables()
# ## Creation of a list of variables with the same shape as the trainable ones
# # initialized with 0s
# accum_vars = [tf.Variable(tf.zeros_like(tv.initialized_value()), trainable=False) for tv in variables_to_train]
# zero_ops = [tv.assign(tf.zeros_like(tv)) for tv in accum_vars]

# ## Calls the compute_gradients function of the optimizer to obtain... the list of gradients
# gvs = optimizer.compute_gradients(total_loss, variables_to_train)

# ## Adds to each element from the list you initialized earlier with zeros its gradient (works because accum_vars and gvs are in the same order)
# accum_ops = [accum_vars[i].assign_add(gv[0]) for i, gv in enumerate(gvs) if gv is not None]

# ## Define the training step (part with variable value update)
# train_step = optimizer.apply_gradients([(tf.multiply(accum_vars[i], accumulate_factor), gv[1]) for i, gv in enumerate(gvs) if gv is not None], global_step=global_step)

# batch_xs = [np.array([[1,2,3],[1,2,3]]),np.array([[1,2,3],[1,2,3]])]
# batch_ys = [np.array([[1,2,3,4],[1,2,3,4]]),np.array([[1,2,3,4],[1,2,3,4]])]
# # Initialize the variables (i.e. assign their default value)
# init = tf.global_variables_initializer()

# # Start training
# with tf.Session() as sess:
#     # Run the initializer
#     sess.run(init)
#     # Loop over all batches
#     for i in range(total_batch):
#         sess.run(zero_ops)
#         # Accumulate the gradients 'n_minibatches' times in accum_vars using accum_ops
#         for i in range(ACCUMULATE_STEP):
#             grad_accumulate, _=sess.run([accum_vars, accum_ops], feed_dict={x: batch_xs[i], y: batch_ys[i]})
#             print(grad_accumulate)
#         _, cur_step = sess.run([train_step, global_step])
#         print(cur_step)

In [8]:
# def my_create_train_op(total_loss, optimizer, summarize_gradients = False):
#     global_step = tf.train.get_or_create_global_step()

#     update_ops = set(ops.get_collection(ops.GraphKeys.UPDATE_OPS))

#     # Make sure update_ops are computed before total_loss.
#     if update_ops:
#         with ops.control_dependencies(update_ops):
#             barrier = control_flow_ops.no_op(name='update_barrier')
#     total_loss = control_flow_ops.with_dependencies([barrier], total_loss)

#     variables_to_train = tf_variables.trainable_variables()

#     # Create the gradients. Note that apply_gradients adds the gradient
#     # computation to the current graph.
#     grads = optimizer.compute_gradients(
#       total_loss,
#       variables_to_train,
#       gate_gradients=tf_optimizer.Optimizer.GATE_OP,
#       aggregation_method=None,
#       colocate_gradients_with_ops=False)

#     # Summarize gradients.
#     if summarize_gradients:
#         with ops.name_scope('summarize_grads'):
#             add_gradients_summaries(grads)

#     # Create gradient updates.
#     grad_updates = optimizer.apply_gradients(grads, global_step=global_step)

#     with ops.name_scope('train_op'):
#         # Make sure total_loss is valid.
#         total_loss = array_ops.check_numerics(total_loss, 'LossTensor is inf or nan')

#     # Ensure the train_tensor computes grad_updates.
#     train_op = control_flow_ops.with_dependencies([grad_updates], total_loss)

#     # Add the operation used for training to the 'train_op' collection
#     train_ops = ops.get_collection_ref(ops.GraphKeys.TRAIN_OP)
#     if train_op not in train_ops:
#         train_ops.append(train_op)

#     return train_op
def my_create_train_op(total_loss, optimizer, summarize_gradients = False, accumulate_factor=None):
    global_step = tf.train.get_or_create_global_step()

    update_ops = set(ops.get_collection(ops.GraphKeys.UPDATE_OPS))

    # Make sure update_ops are computed before total_loss.
    if update_ops:
        with ops.control_dependencies(update_ops):
            barrier = control_flow_ops.no_op(name='update_barrier')
    total_loss = control_flow_ops.with_dependencies([barrier], total_loss)

    variables_to_train = tf_variables.trainable_variables()

    # initialized with 0s
    accum_vars = [tf.Variable(tf.zeros_like(tv.initialized_value()), trainable=False) for tv in variables_to_train]
    zero_ops = [tv.assign(tf.zeros_like(tv)) for tv in accum_vars]

    # Calls the compute_gradients function of the optimizer to obtain... the list of gradients
    grads = optimizer.compute_gradients(
      total_loss,
      variables_to_train,
      gate_gradients=tf_optimizer.Optimizer.GATE_OP,
      aggregation_method=None,
      colocate_gradients_with_ops=False)

    ## Adds to each element from the list you initialized earlier with zeros its gradient (works because accum_vars and grads are in the same order)
    if accumulate_factor is not None:   
        total_loss = array_ops.check_numerics(total_loss, 'LossTensor is inf or nan')
        with tf.control_dependencies([total_loss]):
            accum_ops = [accum_vars[i].assign_add(gv[0]) for i, gv in enumerate(grads) if gv[0] is not None]

        ## Define the training step (part with variable value update)
        accumulate_grads = [(tf.multiply(accum_vars[i], accumulate_factor), gv[1]) for i, gv in enumerate(grads) if gv[0] is not None]
    else:
        accum_ops = tf.no_op(name = 'accum_pass_by')
    
    if accumulate_factor is not None: 
        # Summarize gradients.
        if summarize_gradients:
            with ops.name_scope('summarize_grads'):
                add_gradients_summaries(accumulate_grads)
        grad_updates = optimizer.apply_gradients(accumulate_grads, global_step=global_step)
    else:
        # Summarize gradients.
        if summarize_gradients:
            with ops.name_scope('summarize_grads'):
                add_gradients_summaries(grads)
        grad_updates = optimizer.apply_gradients(grads, global_step=global_step)

    with ops.name_scope('train_op'):
        # Ensure the train_tensor computes grad_updates.
        train_op = control_flow_ops.with_dependencies([grad_updates], total_loss)

    # Add the operation used for training to the 'train_op' collection
    train_ops = ops.get_collection_ref(ops.GraphKeys.TRAIN_OP)
    if train_op not in train_ops:
        train_ops.append(train_op)

    return train_op, accum_ops, zero_ops

In [9]:
def_graph = tf.Graph()
with def_graph.as_default() as graph:
    def train_step(input_examples, one_hot_labels, level0_labels, level1_labels):   
        # inputs has shape [batch, 224, 224, 3]
        with slim.arg_scope(resnet1.resnet_arg_scope()):
            logits, end_points = resnet1.resnet_v1_50(input_examples, NUM_CLASS, is_training=True)
            
            variables_to_restore = slim.get_variables_to_restore(exclude = ['resnet_v1_50/logits'])
            
            level0_aux_input = def_graph.get_tensor_by_name('resnet_v1_50/block3/unit_6/bottleneck_v1/Relu:0')
            level1_aux_input = def_graph.get_tensor_by_name('resnet_v1_50/block4/unit_3/bottleneck_v1/Relu:0')   
#             level0_aux_input = def_graph.get_tensor_by_name('resnet_v1_50/block2/unit_4/bottleneck_v1/Relu:0')
#             level1_aux_input = def_graph.get_tensor_by_name('resnet_v1_50/block3/unit_6/bottleneck_v1/Relu:0')   
           
            level0_aux_net = math_ops.reduce_mean(level0_aux_input, [1, 2], name='level0_aux_pool', keep_dims=True)
            level0_aux_net = layers.conv2d(
                  level0_aux_net,
                  LEVEL0_CLASS, [1, 1],
                  activation_fn=None,
                  normalizer_fn=None,
                  scope='level0_aux_conv')
            level0_aux_logits = tf.squeeze(level0_aux_net)
            loss_level0 = tf.losses.softmax_cross_entropy(onehot_labels = level0_labels, logits = level0_aux_logits, weights=0.02)
            level1_aux_net = math_ops.reduce_mean(level1_aux_input, [1, 2], name='level1_aux_pool', keep_dims=True)
            level1_aux_net = layers.conv2d(
                  level1_aux_net,
                  LEVEL1_CLASS, [1, 1],
                  activation_fn=None,
                  normalizer_fn=None,
                  scope='level1_aux_conv')
            level1_aux_logits = tf.squeeze(level1_aux_net)
            loss_level1 = tf.losses.softmax_cross_entropy(onehot_labels = level1_labels, logits = level1_aux_logits, weights=0.15)

            #variables_to_restore_from_checkpoint = slim.get_variables_to_restore(exclude = variables_to_exclude)
            # Performs the equivalent to tf.nn.sparse_softmax_cross_entropy_with_logits but enhanced, e.x. label smothing
            loss = tf.losses.softmax_cross_entropy(onehot_labels = one_hot_labels, logits = tf.squeeze(logits))
            total_loss = tf.losses.get_total_loss()    # obtain the regularization losses as well
                
#             def wrap_with_counter(fn, counter):
#                 def wrapped_fn(*args, **kwargs):
#                     # control_dependencies forces the assign op to be run even if we don't use the result
#                     with tf.control_dependencies([tf.assign_add(counter, 1)]):
#                         return fn(*args, **kwargs)
#                 return wrapped_fn

#             graph_counter = tf.get_variable(
#                 dtype=tf.int32, shape=(), name='graph_counter',
#                 initializer=tf.zeros_initializer())
#             total_loss = wrap_with_counter(tf.losses.get_total_loss, graph_counter)()
            # Create the global step for monitoring the learning_rate and training.
            # since supervisor will also create one global_step, so we create n advance in order to feed into exponential_decay
            global_step = tf.train.get_or_create_global_step(graph = graph)

            #Define your exponentially decaying learning rate
            lr = tf.train.exponential_decay(
                learning_rate = initial_learning_rate,
                global_step = global_step,
                decay_steps = decay_steps,
                decay_rate = learning_rate_decay_factor,
                staircase = True)

            #Now we can define the optimizer that takes on the learning rate
            optimizer = tf.train.AdamOptimizer(learning_rate = lr)
            #optimizer = tf.train.MomentumOptimizer(learning_rate = lr, momentum=momentum)
            
            # Gather update_ops from the first clone. These contain, for example,
            # the updates for the batch_norm variables created by network_fn.
            # update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

            moving_average_variables = slim.get_model_variables()
            variable_averages = tf.train.ExponentialMovingAverage(moving_average_decay, global_step)
            # Use an alternative set of update ops in addition to the default updates:
            tf.add_to_collection(tf.GraphKeys.UPDATE_OPS, variable_averages.apply(moving_average_variables))

            #Create the train_op.
            accumulate_factor = tf.constant([1./ACCUMULATE_STEP])
            train_op, accum_ops, zero_ops = my_create_train_op(total_loss, optimizer, False, accumulate_factor)
            
            #Create the train_op.
            #train_op = slim.learning.create_train_op(total_loss, optimizer, summarize_gradients=False)

            #State the metrics that you want to predict. We get a predictions that is not one_hot_encoded.
            predictions = tf.argmax(tf.squeeze(end_points['predictions']), 1)
            probabilities = end_points['predictions']
            accuracy, accuracy_update = tf.contrib.metrics.streaming_accuracy(predictions, tf.argmax(one_hot_labels, 1))
            metrics_op = tf.group(accuracy_update)


            #Now finally create all the summaries you need to monitor and group them into one summary op.
            tf.summary.scalar('losses/Total_Loss', total_loss)
            tf.summary.scalar('train/accuracy', accuracy)
            tf.summary.scalar('learning_rate', lr)

            return train_op, accum_ops, zero_ops, global_step, metrics_op, variables_to_restore, predictions, lr, accuracy, total_loss

    def validation_step(input_examples, one_hot_labels):   
        with slim.arg_scope(resnet1.resnet_arg_scope()):
            logits, end_points = resnet1.resnet_v1_50(input_examples, NUM_CLASS, is_training=False, reuse=True)

            #State the metrics that you want to predict. We get a predictions that is not one_hot_encoded.
            predictions = tf.argmax(tf.squeeze(end_points['predictions']), 1)
            probabilities = end_points['predictions']
            accuracy, accuracy_update = tf.contrib.metrics.streaming_accuracy(predictions, tf.argmax(one_hot_labels, 1))
            metrics_op = tf.group(accuracy_update)

            #Now finally create all the summaries you need to monitor and group them into one summary op.
            tf.summary.scalar('validation/accuracy', accuracy)

            return metrics_op, accuracy, predictions, probabilities

In [10]:
with def_graph.as_default() as graph:
    label_mapping = LabelMapping(CATEGORY_NAME_PATH)
    train_dataset = CdiscountDataset(TRAIN_PATH, 'output_file', label_mapping, TOTAL_EXAMPLES, NUM_CLASS, 8000, BATCH_SIZE, NUM_EPOCHES, True)
    val_dataset = CdiscountDataset(VAL_PATH, 'test_output_file', label_mapping, VAL_EXAMPLES, NUM_CLASS, 2000, BATCH_SIZE, VAL_NUM_EPOCHES, False)


    #train_iterator_initializer = train_dataset.create_dataset()
    #val_iterator_initializer = val_dataset.create_dataset()
    #iterator_initalizer = tf.group(train_iterator_initializer, val_iterator_initializer)
    
    #batch_images, batch_labels, batch_level0_labels, batch_level1_labels = train_dataset.get_next()
    #batch_val_images, batch_val_labels, batch_val_level0_labels, batch_val_level1_labels = val_dataset.get_next()
    batch_images, batch_labels, batch_level0_labels, batch_level1_labels = train_dataset.create_dataset()
    batch_val_images, batch_val_labels, batch_val_level0_labels, batch_val_level1_labels = val_dataset.create_dataset()
    
#     batch_images = tf.random_uniform([BATCH_SIZE, 180, 180, 3], maxval=256, dtype=tf.float32)
#     batch_labels = tf.random_uniform([BATCH_SIZE, NUM_CLASS], maxval=1, dtype=tf.int32)
#     batch_level0_labels = tf.random_uniform([BATCH_SIZE, LEVEL0_CLASS], maxval=1, dtype=tf.int32)
#     batch_level1_labels = tf.random_uniform([BATCH_SIZE, LEVEL1_CLASS], maxval=1, dtype=tf.int32)
    with tf.device('/gpu:0'):
        train_op, accum_op, zero_op, global_step, metrics_op, variables_to_restore, pred_op, lr, accuracy, total_loss = train_step(batch_images, batch_labels, batch_level0_labels, batch_level1_labels)
        val_metrics_op, val_accuracy, val_predictions, val_probabilities = validation_step(batch_val_images, batch_val_labels)
        real_val_label = tf.argmax(batch_val_labels, 1)
         
    summary_op = tf.summary.merge_all()
    # Create a saver that restores only the pre-trained variables.
    # we have change optim, restore all param use pretrained mode
    #pre_train_saver = tf.train.Saver(variables_to_restore)
    
    variables = slim.get_variables_to_restore()
    restore_from_pretrained = tf.contrib.framework.filter_variables(
        variables,
        include_patterns=None,
        exclude_patterns=['Momentum'])

    pre_train_saver = tf.train.Saver(variables_to_restore)
    # Define an init function that loads the pretrained checkpoint.
    # sess is the managed session passed by Supervisor
    def load_pretrain(sess):
        pre_train_saver.restore(sess, PRETRAINED_MODEL_PATH)

    # no need for specify local_variables_initializer and tables_initializer, Supervisor will do this via default local_init_op
    # init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer(), tf.tables_initializer())
    init_op = tf.group(tf.global_variables_initializer())
    #init_op = tf.group(train_iterator_initializer, val_iterator_initializer, tf.global_variables_initializer())
    
    # Pass the init function to the supervisor.
    # - The init function is called _after_ the variables have been initialized by running the init_op.
    # - use default tf.Saver() for ordinary save and restore
    # - save checkpoint every 1.3 hours(4800)
    # - manage summary in current process by ourselves for memory saving
    # - no need to specify global_step, supervisor will find this automately
    # - initialize order: checkpoint -> local_init_op -> init_op -> init_func
    sv = tf.train.Supervisor(logdir=LOG_PATH, init_fn = load_pretrain, init_op = init_op, summary_op = None, save_model_secs=8000, checkpoint_basename='resnet50_v1_model.ckpt')
    
    final_loss = 0.
    final_accuracy = 0.
    training_state = True

    config = tf.ConfigProto(log_device_placement=True, allow_soft_placement=True)
    #config.gpu_options.allow_growth = True
    with sv.managed_session(config=config) as sess:
    #with sv.prepare_or_wait_for_session(config=tf.ConfigProto(log_device_placement=True, allow_soft_placement=True)) as sess:
        #sess.run(iterator_initalizer)
        # Here sess was either initialized from the pre-trained-checkpoint or
        # recovered from a checkpoint saved in a previous run of this code.
        for step in range(int(num_steps_per_epoch * NUM_EPOCHES)):         
            if sv.should_stop():
                tf_logging.info('Supervisor emit finished!')
                tf_logging.info('Current Loss: %s', loss)
                tf_logging.info('Current Accuracy: %s', accuracy)
                tf_logging.info('Saving current model to disk(maybe invalid).')
                training_state = False
                break

            start_time = time.time()

            if step % 1000 == 0:
                summ = sess.run(summary_op)
                sv.summary_computed(sess, summ)
                if step > EPOCHES_OVER * num_steps_per_epoch:
                    raise StopIteration("over epoches reached.")

            if step % VAL_CHECK_FREQ == 0:
                with tf.device('/gpu:0'):
                    _, val_acc, val_pred, val_prob, real_label = sess.run([val_metrics_op, val_accuracy, val_predictions, val_probabilities, real_val_label])
                time_elapsed = time.time() - start_time
                tf_logging.info('Validation Speed: {:5.3f}sec/batch'.format(time_elapsed))
                tf_logging.info('Current Streaming ValAccuracy: {:5.3f}%'.format(val_acc*100.))
                tf_logging.info('Real Label: {}'.format(real_label))
                tf_logging.info('Pred Label: {}'.format(val_pred))

            else:
                with tf.device('/gpu:0'):
                    # accumulate gradient to get bigger batch_size
                    sess.run(zero_op)
                    for _ in range(1, ACCUMULATE_STEP):
                        sess.run([accum_op, total_loss])
                    _, _, _, cur_loss, cur_acc, total_step, cur_lr = sess.run([train_op, accum_op, metrics_op, total_loss, accuracy, global_step, lr])
#                     sess.run([train_op])
                time_elapsed = time.time() - start_time

                if step % 10 == 0:
                    final_loss = cur_loss
                    final_accuracy = cur_acc
                    tf_logging.info('Current Speed: {:5.3f}sec/batch'.format(time_elapsed))
                    tf_logging.info('Current Streaming Accuracy: {:5.3f}%'.format(cur_acc*100.))
                    tf_logging.info('Current Loss: {:5.3f}'.format(cur_loss))
                    tf_logging.info('Epoch %s/%s, Global Step: %s', int(total_step / num_steps_per_epoch + 1), NUM_EPOCHES, total_step)
                    tf_logging.info('Current Learning Rate: {}'.format(cur_lr))
                
                    
        if training_state:
            #We log the final training loss and accuracy
            tf_logging.info('Final Loss: %s', final_loss)
            tf_logging.info('Final Accuracy: %s', final_accuracy)
            # Once all the training has been done, save the log files and checkpoint model
            tf_logging.info('Finished training! Model saved.')
        sv.saver.save(sess, sv.save_path, global_step = sv.global_step)
    

INFO:tensorflow:Restoring parameters from /media/rs/0E06CD1706CD0127/Kapok/kaggle/Resnet/logs/resnet50_v1_model.ckpt-0
INFO:tensorflow:Starting standard services.
INFO:tensorflow:Saving checkpoint to path /media/rs/0E06CD1706CD0127/Kapok/kaggle/Resnet/logs/resnet50_v1_model.ckpt
INFO:tensorflow:Starting queue runners.
INFO:tensorflow:global_step/sec: 0
INFO:tensorflow:Validation Speed: 15.441sec/batch
INFO:tensorflow:Current Streaming ValAccuracy: 0.000%
INFO:tensorflow:Real Label: [3692 3692 4350 4393 2938 3797 4002  689 4045  567  991 4393 1435 1409 3277
 2854 4815  796 3168 4350 3663 1010 2647 4814  762 3692 4393 4393 3853 2454
 4350 4045 5090 4350 3277  529 4393 1027 3728 3834 2938 1229 1225 2647 4346
 1444 3186 4350 2850 4286 4429 4350 1862 4350 1401 2311  232 4814 4970 3809
 3795 2884  208 4350 4447 1282 4797  502 1011  289 3182  494  377 3463  377
   59 3185  457 4393 4045 1601 4350 2791 4792 4861 5023 3623 4153   25 3085
 5153 3678  794 2602 5229 1144 1539 4350 4350 4153 4350  

INFO:tensorflow:Pred Label: [1046 3024 1046 3623 2872  223 4814 1046 1046 4814 1046 1046  223 1046 4814
 3182 1046 4951 1046 4951 1046 2872 4951 3623 1046 3623 4951 4787 1046 1046
 4951 3623 3346 1046 2888 3346  223 1046  223 4814 4814 1046 1046 1046 1046
 4787 1046 4787  223 4814 1046 4787 1046 1046 1046 1046 1046 2872 1046 3623
  223  223 1046 4814 4787 3623 4814 1046 4787 3623 1046 4787 4951 4951 4787
 1046 1046 2872 1046 1046 3623 1046 1046 4787  223 4787 2872 1046  223 1046
 1046 1046 1046 3182 1046 4814 3607 1046 1046 2872 1046 1046 3623  223 4951
 3623  223  223 4787 1046 1046 4787 1046  223 1311 1046 3182 1046  223 1046
 1046 4787 4951 3563 1046 1046 1046 1046]
INFO:tensorflow:Current Speed: 1.484sec/batch
INFO:tensorflow:Current Streaming Accuracy: 2.865%
INFO:tensorflow:Current Loss: 10.073
INFO:tensorflow:Epoch 1/8, Global Step: 157
INFO:tensorflow:Current Learning Rate: 9.999999747378752e-05
INFO:tensorflow:Current Speed: 1.571sec/batch
INFO:tensorflow:Current Streaming Acc

INFO:tensorflow:global_step/sec: 0.666717
INFO:tensorflow:Current Speed: 1.434sec/batch
INFO:tensorflow:Current Streaming Accuracy: 4.016%
INFO:tensorflow:Current Loss: 9.855
INFO:tensorflow:Epoch 1/8, Global Step: 314
INFO:tensorflow:Current Learning Rate: 9.999999747378752e-05
INFO:tensorflow:Current Speed: 1.506sec/batch
INFO:tensorflow:Current Streaming Accuracy: 4.068%
INFO:tensorflow:Current Loss: 9.835
INFO:tensorflow:Epoch 1/8, Global Step: 324
INFO:tensorflow:Current Learning Rate: 9.999999747378752e-05
INFO:tensorflow:Current Speed: 1.489sec/batch
INFO:tensorflow:Current Streaming Accuracy: 4.134%
INFO:tensorflow:Current Loss: 9.821
INFO:tensorflow:Epoch 1/8, Global Step: 334
INFO:tensorflow:Current Learning Rate: 9.999999747378752e-05
INFO:tensorflow:Validation Speed: 0.179sec/batch
INFO:tensorflow:Current Streaming ValAccuracy: 0.335%
INFO:tensorflow:Real Label: [ 104 3328 3692 3800 4393 3692 3692 4064 1636 3316 3797 2938  289 3797 2925
 3623  200  328 5218 1005 4429  457 2

INFO:tensorflow:Pred Label: [3929 2938 3929 3929 3929 3929 4970 3929 3623 3929 3929  991 3929  991 3929
 3929 3929 4393 3929 3929 3929 4970 3929 4393 4279 4393 3929 3929 3929 3929
 3929 3929 3929 3929 4814 4970 3929 4814 3623 3929 4970 3929 3330 4814 3929
 3623 3929 3929 4393 3929 3929 3929 4393 4861 4861 3929 3929 3330 3929 3929
 3929 3623 3929 4970 3929 4970 3929 4970 4861  991 4970 3929 3929 3929 3929
 3929 3929 3929 3929 3929  991 3929 3929 3929 3929 3929 3929 4393  991 3929
 3623 3929 3929 3929 4970 4970 3929 3929 4393  991 4970 4861 3929 4970 3929
 3929 3929 4814 3929 3929 3929 3929 3929 4970 3929 4970 3929 3929 3929 3929
 3929 4970 3929 4393 3929 4814 3929 3929]
INFO:tensorflow:Current Speed: 1.515sec/batch
INFO:tensorflow:Current Streaming Accuracy: 4.705%
INFO:tensorflow:Current Loss: 9.769
INFO:tensorflow:Epoch 1/8, Global Step: 500
INFO:tensorflow:Current Learning Rate: 9.999999747378752e-05
INFO:tensorflow:Current Speed: 1.493sec/batch
INFO:tensorflow:Current Streaming Accu

INFO:tensorflow:Current Speed: 1.481sec/batch
INFO:tensorflow:Current Streaming Accuracy: 5.142%
INFO:tensorflow:Current Loss: 9.760
INFO:tensorflow:Epoch 1/8, Global Step: 657
INFO:tensorflow:Current Learning Rate: 9.999999747378752e-05
INFO:tensorflow:Current Speed: 1.529sec/batch
INFO:tensorflow:Current Streaming Accuracy: 5.141%
INFO:tensorflow:Current Loss: 9.769
INFO:tensorflow:Epoch 1/8, Global Step: 667
INFO:tensorflow:Current Learning Rate: 9.999999747378752e-05
INFO:tensorflow:Current Speed: 1.499sec/batch
INFO:tensorflow:Current Streaming Accuracy: 5.138%
INFO:tensorflow:Current Loss: 9.773
INFO:tensorflow:Epoch 1/8, Global Step: 677
INFO:tensorflow:Current Learning Rate: 9.999999747378752e-05
INFO:tensorflow:Validation Speed: 0.200sec/batch
INFO:tensorflow:Current Streaming ValAccuracy: 2.065%
INFO:tensorflow:Real Label: [1435  587 3593 3991 2842  289  991 3328 4350 3672 1453  689 4153 4153 3165
 4350 2897 1301 3929 4160 2611 4772  790 3800 3302  935 1277 2923 4970 4839
 43

...

INFO:tensorflow:Pred Label: [4970 2886 2886 2886 4814 3279 2938 3279 4970  991 2886 4814 4393 4814 4350
 2938 4393  991 4393 4393 4814 4393 3800 3279 2886 2938 3279 3797  991 4814
 4393  991 2886  232 4814 3279  991 4861 4393 4814 4393  232 3279 3800 3279
 4393 4861  991  991 3279 4861  991 3279 4393 2886 2938 3279 1607 3800 4861
 4279 4814 2886 3800 4393 3623 2938 4393 2886  991 4861  991 3800 2886 4393
 3800 3797 4861 3279 4393 2886 3279 4861 3929 1607 2938 4393 4814 4861 4861
  991 2886  991 4393 4393 2938 4861 4393 2886 3797 2938 4861 1372 3623 3279
  991 2938  991  991 4814 1607 2938 3279 4393 4861 4814  991 4393 2938 3279
 3279 2938  991 4393 2938 3279 3279 3623]
INFO:tensorflow:Current Speed: 1.218sec/batch
INFO:tensorflow:Current Streaming Accuracy: 8.696%
INFO:tensorflow:Current Loss: 9.520
INFO:tensorflow:Epoch 2/8, Global Step: 44061
INFO:tensorflow:Current Learning Rate: 2.621440216898918e-05
INFO:tensorflow:Current Speed: 1.278sec/batch
INFO:tensorflow:Current Streaming Ac

INFO:tensorflow:Current Streaming Accuracy: 8.699%
INFO:tensorflow:Current Loss: 9.477
INFO:tensorflow:Epoch 2/8, Global Step: 44218
INFO:tensorflow:Current Learning Rate: 2.621440216898918e-05
INFO:tensorflow:Current Speed: 1.212sec/batch
INFO:tensorflow:Current Streaming Accuracy: 8.699%
INFO:tensorflow:Current Loss: 9.548
INFO:tensorflow:Epoch 2/8, Global Step: 44228
INFO:tensorflow:Current Learning Rate: 2.621440216898918e-05
INFO:tensorflow:Current Speed: 1.243sec/batch
INFO:tensorflow:Current Streaming Accuracy: 8.699%
INFO:tensorflow:Current Loss: 9.494
INFO:tensorflow:Epoch 2/8, Global Step: 44238
INFO:tensorflow:Current Learning Rate: 2.621440216898918e-05
INFO:tensorflow:Validation Speed: 0.152sec/batch
INFO:tensorflow:Current Streaming ValAccuracy: 9.444%
INFO:tensorflow:Real Label: [3277  322 4970 4564 3643  707 3104 2958 2647 4861 3188 4346 3409 1943 4756
 3192 1301 3407   98  314 2950 3188   92  330  689 3059  265 3593 2836 4637
 2878 2761 2938  289 2821 4861 3808 1836  7

INFO:tensorflow:Pred Label: [4393 3279 2886  991 4814 3279 3800  991 3279 4393 4861 1607  991 3279 2938
 4861 4814 4861 3929 3797 3800 3279 3929 3800 3623 3279 4393  991 2938 4393
 4861 3279 3800 3330 4393 2938 3623 2886  991 3279 3797 4350 4350 2886 2886
 4861 4861 3279 3623  991 3797 3279 4814 3800 3279 2938 3330 4814 4861 2886
  991 3797 4970 4393 2886 3279 4814 3279 4279 4350 4861 3797 4861 4970 4393
 3623 3279  991 3797 2938 4350 4279 4393 3929 4393  991 3797 4393 4861 2938
 3929 4350 4393 4814  991 3800 3279 4861 4970  232 4861 4970 1607 3279  991
 2938 4393 3929 2938 3797 4279 3279 4393  991 2938 4393 3929 3623  991 1607
 3279  991 2938 4393 3623  991 3800 4279]
INFO:tensorflow:Current Speed: 1.314sec/batch
INFO:tensorflow:Current Streaming Accuracy: 8.704%
INFO:tensorflow:Current Loss: 9.483
INFO:tensorflow:Epoch 2/8, Global Step: 44404
INFO:tensorflow:Current Learning Rate: 2.621440216898918e-05
INFO:tensorflow:Current Speed: 1.238sec/batch
INFO:tensorflow:Current Streaming Ac

INFO:tensorflow:Current Streaming Accuracy: 8.705%
INFO:tensorflow:Current Loss: 9.405
INFO:tensorflow:Epoch 2/8, Global Step: 44561
INFO:tensorflow:Current Learning Rate: 2.621440216898918e-05
INFO:tensorflow:Current Speed: 1.265sec/batch
INFO:tensorflow:Current Streaming Accuracy: 8.706%
INFO:tensorflow:Current Loss: 9.515
INFO:tensorflow:Epoch 2/8, Global Step: 44571
INFO:tensorflow:Current Learning Rate: 2.621440216898918e-05
INFO:tensorflow:Current Speed: 1.300sec/batch
INFO:tensorflow:Current Streaming Accuracy: 8.706%
INFO:tensorflow:Current Loss: 9.496
INFO:tensorflow:Epoch 2/8, Global Step: 44581
INFO:tensorflow:Current Learning Rate: 2.621440216898918e-05
INFO:tensorflow:Validation Speed: 0.155sec/batch
INFO:tensorflow:Current Streaming ValAccuracy: 9.455%
INFO:tensorflow:Real Label: [3623 3795 1193 2849 4023 3728 2878 3462 3888 1138 4970  302 3277 4957 1625
  207 1282  161 2461 1190 1645 1756 2890 2496 4475 5130 3268 1292  236   98
 3134 4810  504 3545 3040 2841  790 4814 36

KeyboardInterrupt: 

In [ ]:
# with def_graph.as_default() as graph:
#     label_mapping = LabelMapping(CATEGORY_NAME_PATH)
#     train_dataset = CdiscountDataset(TRAIN_PATH, 'output_file', label_mapping, TOTAL_EXAMPLES, NUM_CLASS, 8000, BATCH_SIZE, NUM_EPOCHES, True)
#     val_dataset = CdiscountDataset(VAL_PATH, 'test_output_file', label_mapping, VAL_EXAMPLES, NUM_CLASS, 2000, BATCH_SIZE, VAL_NUM_EPOCHES, False)


#     #train_iterator_initializer = train_dataset.create_dataset()
#     #val_iterator_initializer = val_dataset.create_dataset()
#     #iterator_initalizer = tf.group(train_iterator_initializer, val_iterator_initializer)
    
#     #batch_images, batch_labels, batch_level0_labels, batch_level1_labels = train_dataset.get_next()
#     #batch_val_images, batch_val_labels, batch_val_level0_labels, batch_val_level1_labels = val_dataset.get_next()
#     batch_images, batch_labels, batch_level0_labels, batch_level1_labels = train_dataset.create_dataset()
#     batch_val_images, batch_val_labels, batch_val_level0_labels, batch_val_level1_labels = val_dataset.create_dataset()
    
# #     batch_images = tf.random_uniform([BATCH_SIZE, 180, 180, 3], maxval=256, dtype=tf.float32)
# #     batch_labels = tf.random_uniform([BATCH_SIZE, NUM_CLASS], maxval=1, dtype=tf.int32)
# #     batch_level0_labels = tf.random_uniform([BATCH_SIZE, LEVEL0_CLASS], maxval=1, dtype=tf.int32)
# #     batch_level1_labels = tf.random_uniform([BATCH_SIZE, LEVEL1_CLASS], maxval=1, dtype=tf.int32)
#     with tf.device('/gpu:0'):
#         train_op, accum_op, zero_op, global_step, metrics_op, variables_to_restore, pred_op, lr, accuracy, total_loss = train_step(batch_images, batch_labels, batch_level0_labels, batch_level1_labels)
#         val_metrics_op, val_accuracy, val_predictions, val_probabilities = validation_step(batch_val_images, batch_val_labels)
#         real_val_label = tf.argmax(batch_val_labels, 1)
         
#     summary_op = tf.summary.merge_all()
#     # Create a saver that restores only the pre-trained variables.
#     # we have change optim, restore all param use pretrained mode
#     #pre_train_saver = tf.train.Saver(variables_to_restore)
    
#     variables = slim.get_variables_to_restore()
#     restore_from_pretrained = tf.contrib.framework.filter_variables(
#         variables,
#         include_patterns=None,
#         exclude_patterns=['Momentum'])

#     pre_train_saver = tf.train.Saver(variables_to_restore)
#     # Define an init function that loads the pretrained checkpoint.
#     # sess is the managed session passed by Supervisor
#     def load_pretrain(sess):
#         pre_train_saver.restore(sess, PRETRAINED_MODEL_PATH)

#     # no need for specify local_variables_initializer and tables_initializer, Supervisor will do this via default local_init_op
#     # init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer(), tf.tables_initializer())
#     init_op = tf.group(tf.global_variables_initializer())
#     #init_op = tf.group(train_iterator_initializer, val_iterator_initializer, tf.global_variables_initializer())
    
#     # Pass the init function to the supervisor.
#     # - The init function is called _after_ the variables have been initialized by running the init_op.
#     # - use default tf.Saver() for ordinary save and restore
#     # - save checkpoint every 1.3 hours(4800)
#     # - manage summary in current process by ourselves for memory saving
#     # - no need to specify global_step, supervisor will find this automately
#     # - initialize order: checkpoint -> local_init_op -> init_op -> init_func
#     sv = tf.train.Supervisor(logdir=LOG_PATH, init_fn = load_pretrain, init_op = init_op, summary_op = None, save_model_secs=8000, checkpoint_basename='resnet50_v1_model.ckpt')
    
#     final_loss = 0.
#     final_accuracy = 0.
#     training_state = True

#     config = tf.ConfigProto(log_device_placement=True, allow_soft_placement=True)
#     #config.gpu_options.allow_growth = True
#     with sv.managed_session(config=config) as sess:
#     #with sv.prepare_or_wait_for_session(config=tf.ConfigProto(log_device_placement=True, allow_soft_placement=True)) as sess:
#         #sess.run(iterator_initalizer)
#         # Here sess was either initialized from the pre-trained-checkpoint or
#         # recovered from a checkpoint saved in a previous run of this code.
#         for step in range(int(num_steps_per_epoch * NUM_EPOCHES)):         
#             if sv.should_stop():
#                 tf_logging.info('Supervisor emit finished!')
#                 tf_logging.info('Current Loss: %s', loss)
#                 tf_logging.info('Current Accuracy: %s', accuracy)
#                 tf_logging.info('Saving current model to disk(maybe invalid).')
#                 training_state = False
#                 break

#             start_time = time.time()

#             # accumulate gradient to get bigger batch_size
#             with tf.device('/gpu:0'):
#                 sess.run(zero_op)
#                 for _ in range(ACCUMULATE_STEP):
#                     _,ff=sess.run([accum_op, total_loss])
#                     print(ff)

#             if step % 1000 == 0:
#                 with tf.device('/gpu:0'):
#                     _, _, _, summ = sess.run([train_op, global_step, metrics_op, summary_op])
#                 sv.summary_computed(sess, summ)
#                 if step > EPOCHES_OVER * num_steps_per_epoch:
#                     raise StopIteration("over epoches reached.")
#             else:
#                 if step % VAL_CHECK_FREQ == 0:
#                     with tf.device('/gpu:0'):
#                         _, val_acc, val_pred, val_prob, real_label = sess.run([val_metrics_op, val_accuracy, val_predictions, val_probabilities, real_val_label])
#                     time_elapsed = time.time() - start_time
#                     tf_logging.info('Validation Speed: {:5.3f}sec/batch'.format(time_elapsed))
#                     tf_logging.info('Current Streaming ValAccuracy: {:5.3f}%'.format(val_acc*100.))
#                     tf_logging.info('Real Label: {}'.format(real_label))
#                     tf_logging.info('Pred Label: {}'.format(val_pred))
                        
#                 else:
#                     with tf.device('/gpu:0'):
#                         _, total_step, _, cur_loss, cur_acc, cur_lr = sess.run([train_op, global_step, metrics_op, total_loss, accuracy, lr])
#                     time_elapsed = time.time() - start_time
#                     print(cur_loss)
#                     if step % 10 == 0:
#                         final_loss = cur_loss
#                         final_accuracy = cur_acc
#                         tf_logging.info('Current Speed: {:5.3f}sec/batch'.format(time_elapsed))
#                         tf_logging.info('Current Streaming Accuracy: {:5.3f}%'.format(cur_acc*100.))
#                         tf_logging.info('Current Loss: {:5.3f}'.format(cur_loss))
#                         tf_logging.info('Epoch %s/%s, Global Step: %s', int(total_step / num_steps_per_epoch + 1), NUM_EPOCHES, total_step)
#                         tf_logging.info('Current Learning Rate: {}'.format(cur_lr))
                
                    
#         if training_state:
#             #We log the final training loss and accuracy
#             tf_logging.info('Final Loss: %s', final_loss)
#             tf_logging.info('Final Accuracy: %s', final_accuracy)
#             # Once all the training has been done, save the log files and checkpoint model
#             tf_logging.info('Finished training! Model saved.')
#         sv.saver.save(sess, sv.save_path, global_step = sv.global_step)
    